In [4]:
import os
import csv
import sys
import copy
import logging
import numpy as np
#from stats import *
import pandas as pd
import seaborn as sns
from scipy import stats
#from Layer import Layer
import tensorflow as tf 
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.model_selection import train_test_split, KFold
from sklearn.datasets import fetch_california_housing, make_regression
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [5]:
def write_stats(samples, features, loss, epochs, lr, batch_size, cross_k, r2, mse_train, mae_train, residual_train):
    with open('../data/stats.csv', 'a') as f:
        newrow = [samples, features, loss, epochs, lr, batch_size, cross_k, r2, mse_train, mae_train, residual_train]
        writer = csv.writer(f)
        writer.writerow(newrow)

def fill_dataset(dataframe: DataFrame):
    for column in dataframe:
        if dataframe[column].dtype != 'object':
            dataframe[column] = dataframe[column].fillna(dataframe[column].mean())
    return dataframe

def normalize_dataset(X):
    return tf.keras.utils.normalize(X)

def remove_outliers(X, threshold=7):
    z = np.abs(stats.zscore(X))
    return X[(z<threshold).all(axis=1)][:, 0:-1], X[(z<threshold).all(axis=1)][: ,-1]

def make_dataset(X_data,y_data,k):
    X_data, y_data = remove_outliers(np.concatenate([X_data, y_data], axis=1))
    def gen():
        for train_index, test_index in KFold(k).split(X_data):
            X_train, X_test = X_data[train_index], X_data[test_index]
            XN_train, XN_test = normalize_dataset(X_data[train_index]), normalize_dataset(X_data[test_index])
            y_train, y_test = y_data[train_index], y_data[test_index]
            yield X_train,XN_train,y_train,X_test,XN_test,y_test

    return tf.data.Dataset.from_generator(gen, (tf.double,tf.double,tf.double,tf.double,tf.double,tf.double))


In [6]:
total_columns=["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  
k = 2
epochs = 3000
lr = 0.01
batch_size = 120

#XR, yR = make_regression(n_samples=1000, n_features=2, n_informative=5, noise=50, random_state=5)
#X_train, X_test, y_train, y_test = train_test_split(XR, yR, test_size=0.20, random_state=2)
#columns = ["{0}".format(total_columns[i]) for i in range(XR.shape[1]+1)]
#dataframe = pd.DataFrame(np.concatenate([XR, np.reshape(yR, [-1, 1])], axis=1), columns=columns)

stocks = pd.read_csv("../data/datasets/all_stocks_5yr.csv")
dataframe = pd.DataFrame(stocks)
filter = dataframe["Name"]=="AAPL"
dataframe = dataframe.where(filter).dropna()
data = pd.DataFrame()

data = dataframe.drop('Name', axis=1)
data['target'] = dataframe['open'].fillna(dataframe['open'].mean())-dataframe['close'].fillna(dataframe['close'].mean())
#data['type'] = 1 if dataframe['open']-dataframe['close']>=0 else 0
data.head()

,date,open,high,low,close,volume,target
1259,2013-02-08,67.7142,68.4014,66.8928,67.8542,158168416.0,-0.1400
1260,2013-02-11,68.0714,69.2771,67.6071,68.5614,129029425.0,-0.4900
1261,2013-02-12,68.5014,68.9114,66.8205,66.8428,151829363.0,1.6586
1262,2013-02-13,66.7442,67.6628,66.1742,66.7156,118721995.0,0.0286
1263,2013-02-14,66.3599,67.3771,66.2885,66.6556,88809154.0,-0.2957


In [7]:
vif_data = pd.DataFrame()
vif_data["features"] = data.drop('date', axis=1).columns
vif_data["VIF"] = [variance_inflation_factor(data.drop('date', axis=1).values, i) for i in range(len(data.drop('date', axis=1).columns))]
print(vif_data)

  features           VIF
0     open           inf
1     high  5.060050e+04
2      low  4.511802e+04
3    close           inf
4   volume  2.630125e+00
5   target           inf


C:\Users\Utente\AppData\Roaming\Python\Python39\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\Utente\AppData\Roaming\Python\Python39\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\Utente\AppData\Roaming\Python\Python39\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [10]:
class RNNCell(tf.Module):
    def __init__(self, units, input_dim, output_dim):
        #Initialize weights matrices
        self.W_xh = tf.Variable(shape=[input_dim, output_dim])
        self.W_hh = tf.Variable(shape=[units, units])
        self.W_hy = tf.Variable(shape=[output_dim, units])

        #Initialize hidden state (Memory)
        self.h = tf.zeros([units, 1])
    
    def call(self, x):
        self.h = tf.nn.tanh(self.W_hh * self.h * self.W_xh * x)
        output = self.W_hy * self.h
        return output, self.h        

In [9]:
class MarketAI(tf.Module):
    def __init__(self, epochs):
        self.epochs = epochs